# Part 1: Building a Multi-Agent Resume System 🤖
### GDG Amity Bengaluru Workshop

Welcome! In this series, we will build **ResumeGuide**, a 3-agent system to help B.Tech students craft better resumes using **Google ADK (Agent Development Kit)** with **Vertex AI**.

**The 3 Agents:**
1. **ProfileBot** - Collects your info (Branch, Skills, Projects)
2. **ReviewerBot** - Critiques your resume sections
3. **CoachBot** - Gives industry-specific career advice

**Today's Agenda:**
1. Google ADK + Vertex AI Setup
2. Building ProfileBot with Tools
3. Building ReviewerBot

## 📚 Theory: Why Multi-Agent Systems?

**Single Agent Limitation**: One agent doing everything becomes hard to manage and debug.

**Multi-Agent Benefits:**
- **Separation of Concerns**: Each agent has ONE job (collect info, review, advise)
- **Modularity**: Swap or upgrade agents independently
- **Better Prompts**: Smaller, focused instructions work better than giant ones

**Why Google ADK + Vertex AI?**
- **Official Framework**: Google's ADK is the official agent development kit for Gemini
- **Data Privacy**: Student resume data stays in your GCP project
- **Auto Tool Calling**: ADK handles function calling automatically
- **Production Ready**: Easy deployment to Vertex AI Agent Engine

In [ ]:
# 📦 Setup - Installing Google ADK
%pip install -q -U google-adk google-cloud-aiplatform python-dotenv requests


[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
from utils import Agent, print_box

# ✅ The Agent class auto-detects your service account JSON file in this folder!
# Just make sure your GCP service account JSON (e.g., sc-practice-xxx.json) is present.
#
# 🚀 Now using Google ADK (Agent Development Kit) for:
#    - Automatic tool calling
#    - Built-in session management  
#    - Seamless Vertex AI integration

## 1. 🧑‍💼 Agent 1: ProfileBot
ProfileBot's job is to **collect and store** student information. It uses tools to save data.

In [19]:
# --- STUDENT DATABASE (in production: Firestore/Cloud SQL) ---
STUDENT_PROFILE = {}

# --- TOOLS FOR PROFILEBOT ---
def save_branch(branch: str) -> str:
    """Save the student's engineering branch (e.g., CSE, ECE, Mechanical)."""
    STUDENT_PROFILE['branch'] = branch
    return f"✅ Branch saved: {branch}"

def save_skills(skills: str) -> str:
    """Save the student's technical skills as a comma-separated string."""
    STUDENT_PROFILE['skills'] = [s.strip() for s in skills.split(',')]
    return f"✅ Skills saved: {STUDENT_PROFILE['skills']}"

def save_project(title: str, description: str) -> str:
    """Save a project with title and brief description."""
    if 'projects' not in STUDENT_PROFILE:
        STUDENT_PROFILE['projects'] = []
    STUDENT_PROFILE['projects'].append({'title': title, 'desc': description})
    return f"✅ Project added: {title}"

def get_profile() -> str:
    """Retrieve the current student profile."""
    return str(STUDENT_PROFILE) if STUDENT_PROFILE else "No profile data yet."

# --- AGENT 1: PROFILEBOT ---
profile_bot = Agent(
    name="ProfileBot",
    instruction="""You are ProfileBot. Your ONLY job is to collect student info for their resume.
    Ask about: Branch, Skills, Projects. Use the tools to SAVE each piece of info.
    Be friendly and conversational. One question at a time.""",
    tools=[save_branch, save_skills, save_project, get_profile]
)

response = profile_bot.chat("Hi! I'm a 3rd year student looking to build my resume.")
print_box("ProfileBot", response)

🔌 Connecting to Vertex AI (Project: sc-practice-66d-20250731, Loc: us-central1)...
🔑 Using ADC with: service_account.json
⏳ ProfileBot is thinking...

========== ProfileBot ==========
❌ Vertex AI Error: [Errno 54] Connection reset by peer



### ProfileBot in Action
Watch how ProfileBot uses tools to save information as you chat.

In [20]:
# Continue the conversation - provide branch info
response = profile_bot.chat("I'm studying Computer Science Engineering")
print_box("ProfileBot", response)
print(f"\n📊 [Database State]: {STUDENT_PROFILE}")

⏳ ProfileBot is thinking...

========== ProfileBot ==========
Hey there! Great to hear you're getting a head start on your resume. A strong resume can really open doors.

To start, let's save your branch. I see you're in Computer Science Engineering. Is that right?



📊 [Database State]: {}


### Adding Skills & Projects

In [21]:
# Add skills
response = profile_bot.chat("I know Python, React, and some Machine Learning basics")
print_box("ProfileBot", response)
print(f"\n📊 [Database State]: {STUDENT_PROFILE}")

⏳ ProfileBot is thinking...

========== ProfileBot ==========
Awesome! That's a solid skillset. Let's get those skills saved. Just to confirm, your skills are Python, React, and Machine Learning basics, correct?



📊 [Database State]: {}


## 2. 📝 Agent 2: ReviewerBot
ReviewerBot's job is to **analyze resume content** and provide actionable feedback.

In [22]:
# --- TOOLS FOR REVIEWERBOT ---
import requests

def check_skill_demand(skill: str) -> str:
    """Check if a skill is in-demand using a public jobs API (simulated)."""
    # In production: Call LinkedIn API or job boards
    HOT_SKILLS = ["python", "react", "machine learning", "aws", "docker", "kubernetes"]
    if skill.lower() in HOT_SKILLS:
        return f"🔥 '{skill}' is HIGH DEMAND in 2024-25!"
    return f"'{skill}' is useful but consider adding trending skills."

def analyze_project(title: str, description: str) -> str:
    """Analyze a project description for resume quality."""
    issues = []
    if len(description) < 20:
        issues.append("Too short - add more details")
    if not any(word in description.lower() for word in ["built", "developed", "created", "implemented"]):
        issues.append("Use action verbs (built, developed, created)")
    if not any(char.isdigit() for char in description):
        issues.append("Add metrics/numbers (e.g., '50% faster', '1000 users')")
    
    if issues:
        return f"⚠️ Issues: {', '.join(issues)}"
    return "✅ Project description looks strong!"

# --- AGENT 2: REVIEWERBOT ---
reviewer_bot = Agent(
    name="ReviewerBot",
    instruction="""You are ReviewerBot, a resume expert. Your job:
    1. Check if skills are in-demand using `check_skill_demand`
    2. Analyze project descriptions using `analyze_project`
    Give specific, actionable feedback. Be encouraging but honest.""",
    tools=[check_skill_demand, analyze_project]
)

🔌 Connecting to Vertex AI (Project: sc-practice-66d-20250731, Loc: us-central1)...
🔑 Using ADC with: service_account.json


### ReviewerBot in Action

In [23]:
# Review skills
response = reviewer_bot.chat("Can you check if Python and React are good skills to have?")
print_box("ReviewerBot", response)

⏳ ReviewerBot is thinking...

========== ReviewerBot ==========
Okay! It looks like both Python and React are in high demand right now, so definitely highlight those on your resume!




In [24]:
# Review a project description
response = reviewer_bot.chat("""
Review this project for my resume:
Title: E-commerce Website
Description: Made a shopping website using React
""")
print_box("ReviewerBot", response)


⏳ ReviewerBot is thinking...

========== ReviewerBot ==========
Okay, here's my feedback on your E-commerce Website project:

*   **Use stronger action verbs:** Instead of "Made," try "Built," "Developed," or "Created."
*   **Add metrics/numbers:** Quantify your impact. For example, did you improve site performance? How many users does the site have? Did you increase sales? Even estimates can help. Something like "Developed a shopping website using React that served 1000+ users" is much more impactful.




## 3. 🎯 Agent 3: CoachBot (Preview)
CoachBot will provide **industry-specific career advice**. We'll build it fully in Part 2.

**CoachBot's Tools (coming next):**
- `get_industry_trends(branch)` - Fetch trends for CSE, ECE, etc.
- `suggest_certifications(skills)` - Recommend relevant certs

---

## ✅ Part 1 Summary

| Agent | Role | Tools |
|-------|------|-------|
| **ProfileBot** | Collect student info | save_branch, save_skills, save_project |
| **ReviewerBot** | Analyze resume quality | check_skill_demand, analyze_project |
| **CoachBot** | Career guidance | (Part 2) |

**Key Learnings:**
- Each agent has a focused instruction and specific tools
- Tools are just Python functions with docstrings
- **Google ADK** handles the function calling automatically!
- Vertex AI provides enterprise-grade infrastructure

**Next:** Part 2 - Memory & Guardrails (making CoachBot remember context and stay safe)
